# Query the live aircraft data from Opensky API

This notebook shows how to query live data from the Opensky API.
https://opensky-network.org/apidoc/

In [1]:
# Import the libraries that will be used in this notebook
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os

## API configuration and query

In [2]:
# Save config information.
url = "https://opensky-network.org/api/states/all"

# Build partial query URL
query_url = f"{url}"

#  Perform a request for data
response = requests.get(query_url).json()

In [3]:
# Time of the query data
response["time"]

1607061890

In [4]:
# Amount of data point of the query data
len(response["states"])

2191

In [5]:
# One element of the array
response["states"][0]

['aa9321',
 '',
 'United States',
 1607061889,
 1607061889,
 -15.3513,
 54.1372,
 11887.2,
 False,
 283.37,
 123.37,
 0,
 None,
 11498.58,
 None,
 False,
 0]

In [6]:
# Map data to the dictorary
aircraft_live_data = [];
for i in range(len(response["states"])):
    data = {};
    # Test for null and strip spaces in case not null
    if response["states"][i][0]:
        data["icao24"] = response["states"][i][0].strip()
    if response["states"][i][1]:
        data["callsign"] = response["states"][i][1].strip()
    if response["states"][i][2]:
        data["origin_country"] = response["states"][i][2].strip()
    data["time_position"] = response["states"][i][3]
    data["last_contact"] = response["states"][i][4]
    data["longitude"] = response["states"][i][5]
    data["latitude"] = response["states"][i][6]
    data["baro_altitude"] = response["states"][i][7]
    data["on_ground"] = response["states"][i][8]
    data["velocity"] = response["states"][i][9]
    data["true_track"] = response["states"][i][10]
    data["vertical_rate"] = response["states"][i][11]
    data["sensors"] = response["states"][i][12]
    data["geo_altitude"] = response["states"][i][13]
    if response["states"][i][14]:
        data["squawk"] = response["states"][i][14].strip()
    data["spi"] = response["states"][i][15]
    data["position_source"] = response["states"][i][16]
    
    aircraft_live_data.append(data)

In [7]:
aircraft_live_data[0]

{'icao24': 'aa9321',
 'origin_country': 'United States',
 'time_position': 1607061889,
 'last_contact': 1607061889,
 'longitude': -15.3513,
 'latitude': 54.1372,
 'baro_altitude': 11887.2,
 'on_ground': False,
 'velocity': 283.37,
 'true_track': 123.37,
 'vertical_rate': 0,
 'sensors': None,
 'geo_altitude': 11498.58,
 'spi': False,
 'position_source': 0}

## Save data to MongoDB

In [9]:
import pymongo

# Use flask_pymongo to set up mongo connection
conn =  "mongodb://localhost:27017/project_3"
client =  pymongo.MongoClient(conn)

# identify the collection and drop any existing data for this demonstration
db = client.project_3
# db.live_aircraft_data.drop()

db.live_aircraft_data.insert_many(aircraft_live_data)


# query_result = list(db.mars_data.find())
# query_result = (db.mars_data.find())
query_result = (db.live_aircraft_data.find_one())
query_result